<a href="https://colab.research.google.com/github/avoss22/NGG6050/blob/main/QNC_Model_Fitting_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QNC Fit Model to Data Exercise (LATER Data)

## Anna Voss | Due: 10-11-23

## QNC Course Fall 2023

Shoutout ChatGPT for translating the MatLab code in Josh's tutorials to Python: https://chat.openai.com/share/31d43f71-f845-4465-805e-c8af4a099c82

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
#Define the function later_getData
import numpy as np
import os
import scipy.io

def later_getData(subjectTag='', dataDirectory='', expressCutoff=0.0):
    # Data directory
    dataDirectory = os.path.expanduser(dataDirectory)

    # Load the data from the subject
    data = scipy.io.loadmat(os.path.join(dataDirectory, 'data_mgl', 'F', subjectTag + '_RT.mat'))

    # Extract relevant variables from the loaded data
    decisionSum = data['decisionSum'][0]
    labelSum = data['labelSum'][0]
    numdirSum = data['numdirSum'][0]
    percorrSum = data['percorrSum'][0]
    tRxnSum = data['tRxnSum'][0]

    # Define selection criteria
    Ltrials = (percorrSum == 1) & (tRxnSum > expressCutoff) & (tRxnSum < 1.2)

    # Create four data sets
    C_L_0 = tRxnSum[Ltrials & (numdirSum == -1) & (labelSum == 1)]
    C_L_1_plus = tRxnSum[Ltrials & (numdirSum == -1) & (labelSum != 1)]
    C_R_0 = tRxnSum[Ltrials & (numdirSum == 1) & (labelSum == 1)]
    C_R_1_plus = tRxnSum[Ltrials & (numdirSum == 1) & (labelSum != 1)]

    data_ = [C_L_0, C_L_1_plus, C_R_0, C_R_1_plus]

    labels_ = ['Left Choice, CP', 'Left Choice, No CP', 'Right Choice, CP', 'Right Choice, No CP']

    return data_, labels_




1. Get the data

Use this code to get a data set (array of RTs from a single condition) to fit, already preprocessed to include correct trials only and removeoutliers (including express saccades). See later_getData for details

In [21]:
# Call the later_getData function with the appropriate arguments
data = later_getData(subjectTag='FT', dataDirectory='/content/gdrive/MyDrive/LATERdata', expressCutoff=0.2)

# Extract the first element of the data list as RTs
RTs = data[0]

# There's no need to clear data as Python handles memory management automatically

2. Define the objective function

The objective function typically defines the error that you want to minimize between your data and the model predictions. A common objective function is the negative of the sum of the log-likelihoods of the data, given the model parameters. To unpack that for the LATER model:
1. For each data point (RT from a single trial, in this case) and given a set of model parameters, compute the probability of the data, given the model (i.e., the likelihood)
2. Take the logarithm
3. Sum all these log-likelihoods from all the data points
4. Take the negative, because we want to find the minimum (thus corresponding to the maximum likelihood)

In [18]:
import numpy as np
from scipy.stats import norm  # You may need to adjust the distribution based on your model

def later_model_objective(params, data):
    """
    Compute the negative log-likelihood of the data given the model parameters.

    Parameters:
    - params (list or array): Model parameters to be optimized.
    - data (array): Observed reaction times (RTs) from experimental data.

    Returns:
    - neg_log_likelihood (float): Negative log-likelihood of the data.
    """

    # Extract model parameters
    # You would define your specific model parameters here based on your LATER model
    # Example parameters: mu (mean), sigma (standard deviation), etc.
    mu, sigma = params

    # Compute the negative log-likelihood
    log_likelihood = 0.0

    for rt in data:
        # Calculate the probability density function (PDF) of the normal distribution
        pdf = norm.pdf(rt, loc=mu, scale=sigma)

        # Add the log-likelihood of this data point to the total log-likelihood
        log_likelihood += np.log(pdf)

    # Take the negative log-likelihood, because we want to minimize it
    neg_log_likelihood = -log_likelihood

    return neg_log_likelihood


3. Define initial conditions

For the actual fitting, we will use fmincon (https://www.mathworks.com/help/optim/ug/fmincon.html), which is "function minimization with constraints." This function allows for constraints that include upper and lower bounds on the parameters. So here we define those bounds, along with the initial values.

We'll use fairly arbitrary values for the lower and upper bounds, but we should pick the initial values more judiciously. HINT: Recall that the muR and deltaS should be strongly related to empirical summary statistics of `the (reciprocal) RT distribution.

In [24]:
import numpy as np

# Define the lower and upper bounds
lowerBounds = [0.001, 0.001]
upperBounds = [1000, 1000]

# Calculate initial values based on empirical summary statistics (you should replace these with appropriate values)
# For example, you can use the mean and standard deviation of the data
mean_RT = np.mean(data)  # Replace 'data' with your actual RT data
std_RT = np.std(data)  # Replace 'data' with your actual RT data

# Set initial values
initial_values = [mean_RT, std_RT]  # Replace with your calculated values

# Ensure initial values are within bounds
initial_values = np.maximum(lowerBounds, np.minimum(upperBounds, initial_values))

# Now you can use initial_values as the initial guess for your optimization


ValueError: ignored

To Josh: My BFF ChatGPT did well in translating MatLab to Python up to this point. I had to mess around with the getData function at the beginning, so maybe this is why it doesn't like the shape of my matrix, and I'm not getting all of the metadata I need in order to compute initial values. I am stumped here.

4. Run the fits

We will be using GlobalSearch. The general advantage of this approach is to avoid local minima; for details, see: https://www.mathworks.com/help/gads/how-globalsearch-and-multistart-work.html

These options seem to work well, but I don't have a stronger rationale for using them. See the Matlab documentation if you really want to dive in and understand them, and let me know if you find better settings!
opts = optimoptions(@fmincon,    ... % "function minimization with constraints"
   'Algorithm',   'active-set',  ...
   'MaxIter',     3000,          ...
   'MaxFunEvals', 3000);

Definine the "optimization problem" using variables defined above
problem = createOptimProblem('fmincon',    ...
    'objective',   laterErrFcn,     ... % Use the objective function
    'x0',          initialValues,   ... % Initial conditions
    'lb',          lowerBounds,     ... % Parameter lower bounds
    'ub',          upperBounds,     ... % Parameter upper bounds
    'options',     opts);                % Options defined above

Create a GlobalSearch object
gs = GlobalSearch;
   
Run it, returning the best-fitting parameter values and the negative-log-likelihood returned by the objective function
[fits(ii,:), nllk] = run(gs,problem);

In [28]:
import numpy as np
from scipy.optimize import Bounds, differential_evolution

# Define the objective function (laterErrFcn in MATLAB)
def objective_function(params):
    # Calculate the negative log-likelihood using your LATER model
    neg_log_likelihood = later_model_objective(params, data)  # Use your objective function here
    return neg_log_likelihood

# Define parameter bounds (lower and upper bounds)
lower_bounds = [0.001, 0.001]
upper_bounds = [1000, 1000]

# Define bounds for the optimization
bounds = Bounds(lower_bounds, upper_bounds)

# Set initial conditions based on empirical statistics (replace with appropriate values)
mean_RT = np.mean(data)  # Replace 'data' with your actual RT data
std_RT = np.std(data)  # Replace 'data' with your actual RT data
initial_values = [mean_RT, std_RT]  # Replace with your calculated values

# Run global optimization using Differential Evolution
result = differential_evolution(objective_function, bounds, maxiter=3000, popsize=30)

# Extract the best-fitting parameter values and the minimum negative log-likelihood
best_params = result.x
best_neg_log_likelihood = result.fun

# 'best_params' contains the best-fitting parameter values
# 'best_neg_log_likelihood' contains the minimum negative log-likelihood


ValueError: ignored

 5. Evaluate the fits

EXERCISE: How do you know if you got a reasonable answer?

To evaluate if you got a reasonable answer, you could calculate mean squared errors and perform residual analysis to further determne the goodness of fit of the model to the data.